In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import boto3
import datetime
import pandas as pd
import csv
#import io

In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

# assign the usecase variable for writing csv files
usecase_set_1 = ['511','512','514']
usecase_set_2 = ['513']

In [0]:
# get s3 params from metadata table 
s3_query = f"SELECT success_filename_prefix,error_filename_prefix,s3_bucket_name,s3_STG_path,s3_archive_path,Success_Query,Success_Header,Error_Query,Error_Header,s3_ATL_bucket_name,s3_ATL_STG_path,s3_ATL_archive_path,ATL_Error_Query,ATL_Error_Query_Header,ATL_error_filename_prefix from com_us_alyt_ngebox.metadata_usecase where USECASE_ID = '{usecase_id}'"

# assign the value to variable from the result
s3_bucket_query_df = sqlContext.sql(s3_query).collect()[0]['s3_bucket_name']
s3_STG_path_query_df = sqlContext.sql(s3_query).collect()[0]['s3_STG_path']
s3_archive_path_query_df = sqlContext.sql(s3_query).collect()[0]['s3_archive_path']
s3_success_prefix_df = sqlContext.sql(s3_query).collect()[0]['success_filename_prefix']
s3_error_prefix_df = sqlContext.sql(s3_query).collect()[0]['error_filename_prefix']
Success_Query = sqlContext.sql(s3_query).collect()[0]['Success_Query']
Success_Header = sqlContext.sql(s3_query).collect()[0]['Success_Header']
Error_Query = sqlContext.sql(s3_query).collect()[0]['Error_Query']
Error_Header = sqlContext.sql(s3_query).collect()[0]['Error_Header']
s3_ATL_bucket_name_df = sqlContext.sql(s3_query).collect()[0]['s3_ATL_bucket_name']
s3_ATL_STG_path_df = sqlContext.sql(s3_query).collect()[0]['s3_ATL_STG_path']
s3_ATL_archive_path_df = sqlContext.sql(s3_query).collect()[0]['s3_ATL_archive_path']
ATL_Error_Query = sqlContext.sql(s3_query).collect()[0]['ATL_Error_Query']
ATL_Error_Query_Header = sqlContext.sql(s3_query).collect()[0]['ATL_Error_Query_Header']
ATL_error_filename_prefix_df = sqlContext.sql(s3_query).collect()[0]['ATL_error_filename_prefix']

#convert header string to list by assigning index 
Success_Header_list = Success_Header.split(',')
Error_Header_list = Error_Header.split(',')
if usecase_id == '514':
    ATL_Error_Query_Header_list = ATL_Error_Query_Header.split(',')

# Set the S3 bucket and folder path
s3_bucket = s3_bucket_query_df
ATL_s3_bucket = s3_ATL_bucket_name_df
s3_folder_path_STG = s3_STG_path_query_df
ATL_s3_folder_path_STG = s3_ATL_STG_path_df
s3_folder_path_arc = s3_archive_path_query_df
ATL_s3_folder_path_arc = s3_ATL_archive_path_df
s3_success_filename = s3_success_prefix_df
s3_error_filename = s3_error_prefix_df
ATL_error_filename = ATL_error_filename_prefix_df

# Get the current date and time
folder_name = datetime.datetime.now().strftime("%Y-%m-%d")
folder_name_ATL = datetime.datetime.now().strftime("%Y%m%d")

  File "<command-3707175531616020>", line 2
    if usecase_id = 512:
                  ^
SyntaxError: invalid syntax


In [0]:
# Populate success data and the headers
s_query = f"{Success_Query}"
success_query = s_query
headers_sucess = Success_Header_list
success_query_df = sqlContext.sql(success_query).collect()
if len(success_query_df) > 0:
    # Create a list to store the data
    success_list = []
    for curr_row in success_query_df:
        success_list.append(curr_row.asDict(True))
    # Create the DataFrame with data and headers
    success_df = pd.DataFrame(success_list)
else:
    # Create an empty DataFrame with headers
    success_df = pd.DataFrame(columns=headers_sucess)

In [0]:
# Populate error data and the headers
e_query = f"{Error_Query}"
error_query = e_query
headers_error = Error_Header_list
error_query_df = sqlContext.sql(error_query).collect()
if len(error_query_df) > 0:
    # Create a list to store the data
    error_list = []
    for curr_row in error_query_df:
        error_list.append(curr_row.asDict(True))
    # Create the DataFrame with data and headers
    error_df = pd.DataFrame(error_list)
else:
    # Create an empty DataFrame with headers
    error_df = pd.DataFrame(columns=headers_error)

In [0]:
# populate ATL data for error records
if usecase_id == '514':
    atl_query = f"{ATL_Error_Query}"
    error_atl_query = atl_query
    error_atl_header = ATL_Error_Query_Header_list
    error_atl_df = sqlContext.sql(error_atl_query).collect()
    if len(error_atl_df) > 0:
        # Create a list to store the data
        error_list_atl = []
        for curr_row in error_atl_df:
            error_list_atl.append(curr_row.asDict(True))
        # Create the DataFrame with data and headers
        atl_df = pd.DataFrame(error_list_atl)
    else:
        # Create an empty DataFrame with headers
        atl_df = pd.DataFrame(columns=error_atl_header)

In [0]:
# Create the S3 client
s3_client = boto3.client('s3')

# Create the CSV file name using the current date
now = datetime.datetime.now().strftime("%m%d%Y_%H%M%S")
success_csv_file_name = f"{s3_success_filename}{now}.csv"	
error_csv_file_name = f"{s3_error_filename}{now}.csv"
atl_error_file_name = f"{ATL_error_filename}.csv"

# Initialize success_csv_data as an empty string
success_csv_data = ""
if usecase_id in usecase_set_1:
    success_csv_data = success_df.to_csv(index=False)
elif usecase_id in usecase_set_2:
    # Write the header without quoting
    header_csv_data = success_df.columns.to_list()
    header_csv_str = ','.join(header_csv_data) + '\n'  # Convert header to a CSV string
    # Write the data with quoting
    data_csv_data = success_df.to_csv(index=False, sep=',', quoting=csv.QUOTE_ALL, header=False)
    # Combine the header and data
    success_csv_data = header_csv_str + data_csv_data

# Common error_csv_data line for all usecase
error_csv_data = error_df.to_csv(index=False)   

# Write the dynamic CSV file to the S3 bucket folder for success and error files
s3_client.put_object(Bucket=s3_bucket, Key=s3_folder_path_arc + folder_name + '/' + success_csv_file_name, Body=success_csv_data)
s3_client.put_object(Bucket=s3_bucket, Key=s3_folder_path_arc + folder_name + '/' + error_csv_file_name, Body=error_csv_data)

# Remove the existing CSV files in the STG path
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_folder_path_STG)
for obj in response.get('Contents', []):
    if obj['Key'].endswith('.csv'):
        s3_client.delete_object(Bucket=s3_bucket, Key=obj['Key'])
        print(f"Deleted object: {obj['Key']}")

# Copy the new CSV file to the STG path
s3_client.put_object(Bucket=s3_bucket, Key=s3_folder_path_STG + success_csv_file_name, Body=success_csv_data)

if usecase_id == '514':
    # ATL error data 
    atl_csv_data = atl_df.to_csv(index=False)

# Write the dynamic CSV file to the ATL S3 bucket folder for success and error files
if usecase_id == '514':
    # DSOC ATL archile file load
    s3_client.put_object(Bucket=ATL_s3_bucket, Key=ATL_s3_folder_path_arc + folder_name_ATL + '/' + atl_error_file_name, Body=atl_csv_data)
    # remove file from DSOC ATL STG path
    response = s3_client.list_objects_v2(Bucket=ATL_s3_bucket, Prefix=ATL_s3_folder_path_STG)
    for obj in response.get('Contents', []):
        if obj['Key'].endswith('.csv'):
            s3_client.delete_object(Bucket=ATL_s3_bucket, Key=obj['Key'])
            print(f"Deleted object: {obj['Key']}")
    # DSOC ATL STG file load
    s3_client.put_object(Bucket=ATL_s3_bucket, Key=ATL_s3_folder_path_STG + atl_error_file_name, Body= atl_csv_data)

In [0]:
class s3Load:
   def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": "s3 file load completed"
      }
    }
    self.response = resp

In [0]:
metadata_obj = s3Load()
metadata_obj.construct_microservice_response()
dbutils.notebook.exit(
  metadata_obj.response
)